In [0]:
%sql
USE CATALOG hive_metastore;
USE SCHEMA default;


In [0]:
CREATE OR REPLACE TABLE patient_dim
USING DELTA 
LOCATION 's3a://mini-project-sathwik/gold/patient_dim'
AS
SELECT
    -- Surrogate Key
    ROW_NUMBER() OVER (ORDER BY patient_id) AS patient_sk,
    -- Business Key
    patient_id,

    -- Attributes
    first_name,
    last_name,
    name,
    age,
    address,
    phone_number,
    bill_amount,
    insurance_provider,
    policy_id,

    -- SCD-2 Fields
    start_date,
    end_date,

    CASE WHEN end_date = '9999-12-31' THEN 'Y' ELSE 'N' END AS is_current,
    activation_ind,

    -- Audit
    ingest_time,
    source_file,
    create_timestamp,
    update_timestamp,
    load_ctl_key,
    checksum_txt

FROM patient_final_silver;


In [0]:
create or replace view patient_dim_view AS
SELECT * FROM patient_dim;

In [0]:
select * from patient_dim_view;

In [0]:
%sql
CREATE OR REPLACE TABLE insurance_dim
USING DELTA 
AS
SELECT
    -- Surrogate Key
   ROW_NUMBER() OVER (ORDER BY policy_id) AS insurance_sk
,

    -- Business Key
    policy_id,

    -- Attributes
    insurance_provider,
    amount_covered,
    claim_status,
    -- SCD-2 Fields
    start_date,
    end_date,
    CASE WHEN end_date = '9999-12-31' THEN 'Y' ELSE 'N' END AS is_current,
    activation_ind,

    -- Audit / Lineage
    ingest_time,
    source_file,
    create_timestamp,
    update_timestamp,
    load_ctl_key,
    checksum_txt

FROM insurance_final_silver;


In [0]:
create or replace view insurance_dim_view as 
select * from insurance_dim;

In [0]:
select * from insurance_dim_view;

In [0]:
-- SELECT patient_id, is_current, effective_start_dt, effective_end_dt
-- FROM patient_dim
-- ORDER BY patient_id, effective_start_dt;


In [0]:
-- SELECT policy_id, is_current, effective_start_dt, effective_end_dt
-- FROM insurance_dim
-- ORDER BY policy_id, effective_start_dt;